In [1]:
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score
import torch
import numpy as np
from simulator import Simulator
import yaml

setting = yaml.safe_load(open('../configurations/MNIST/MLP/ClusterAgg/average/no_straggle.yaml'))

sim = Simulator(**setting)
n_epoch = 100

In [2]:
for epoch in tqdm(
    range(n_epoch), 
    desc=f'Running {str(sim.output_dir)}', 
    leave=True,
):
    picked_clients, to_update_global = sim.step()
    avg_losses = [u.avg_loss for u in to_update_global]
    train_acc_scores = [u.train_acc_score for u in to_update_global]
    test_acc_scores = [u.test_acc_score for u in to_update_global]

    print(f'Simulator -- got {len(to_update_global)} updates to incorporate')

    nan_in_model = torch.stack([w.isnan().any() for w in sim.global_model.get_state().values()]).any().item()
    if nan_in_model:
        print('nan is bleeding in!!!!')
    
    if len(to_update_global):
        # only update the global model if we have any updates
        avg_train_acc = sum(train_acc_scores) / len(train_acc_scores)
        avg_test_acc = sum(test_acc_scores) / len(test_acc_scores)
        print(f'Simulator -- avg loss: {sum(avg_losses) / len(to_update_global)}')
        print(f'Simulator -- client avg train acc: {sum(train_acc_scores) / len(train_acc_scores)}')
        print(f'Simulator -- client avg test acc: {sum(test_acc_scores) / len(test_acc_scores)}')

        # Step 4. Update the global model with the finished local updates
        new_state = sim.aggregator(sim.global_model, to_update_global)

        if new_state is not None:
            sim.global_model.set_state(new_state)
    else:
        avg_train_acc = 0
        avg_test_acc = 0
    
    pred = sim.global_model.predict(sim.x_test)
    print(f'Simulator -- Global model test acc: {accuracy_score(sim.y_test, pred)}')

Running output/MNIST/MLP/ClusterAgg/average/no_straggle:   0%|          | 0/100 [00:00<?, ?it/s]

Simulator -- got 20 updates to incorporate
Simulator -- avg loss: 2.3208935976028444
Simulator -- client avg train acc: 0.7166666666666667
Simulator -- client avg test acc: 0.6640898345153664
Simulator -- Global model test acc: 0.46335697399527187
Simulator -- got 20 updates to incorporate
Simulator -- avg loss: 1.5979795455932617
Simulator -- client avg train acc: 0.4499999999999999
Simulator -- client avg test acc: 0.4633569739952719
Simulator -- Global model test acc: 0.46335697399527187
Simulator -- got 20 updates to incorporate
Simulator -- avg loss: 1.5979795455932617
Simulator -- client avg train acc: 0.5125
Simulator -- client avg test acc: 0.4633569739952719


/Users/inwon/miniconda3/envs/robust-fl/lib/python3.10/site-packages/sklearn/cluster/_affinity_propagation.py:143: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


Simulator -- Global model test acc: 0.46335697399527187
Simulator -- got 20 updates to incorporate
Simulator -- avg loss: 1.5979795455932617
Simulator -- client avg train acc: 0.45
Simulator -- client avg test acc: 0.4633569739952719
Simulator -- Global model test acc: 0.46335697399527187
Simulator -- got 20 updates to incorporate
Simulator -- avg loss: 1.5979795455932617
Simulator -- client avg train acc: 0.4833333333333333
Simulator -- client avg test acc: 0.4633569739952719


/Users/inwon/miniconda3/envs/robust-fl/lib/python3.10/site-packages/sklearn/cluster/_affinity_propagation.py:143: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


Simulator -- Global model test acc: 0.4491725768321513
Simulator -- got 20 updates to incorporate
Simulator -- avg loss: nan
Simulator -- client avg train acc: 0.4541666666666666
Simulator -- client avg test acc: 0.4633569739952719


ValueError: Input contains NaN.

In [4]:
cids = [c.client_id for c in to_update_global]
nan_clients = [c for c in sim.clients if c.id(cids)]

TypeError: 'int' object is not iterable